<a href="https://colab.research.google.com/github/glgunderson/INFOB2DA-PA4/blob/main/pa4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dashboard Visualizations and Coordinated View Systems**
## Practical Assignment 4 - INFOB2DA
*Tobias Buiten & Grace Gunderson*


In [ ]:
# Import Relevant Libraries for Level 2 Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
# Download Dataset from GitHub Release & Unzip Large Dataset
!curl -L -o DelayedFlights.zip "https://github.com/glgunderson/INFOB2DA-PA4/releases/download/PA4.DATA/DelayedFlights.zip"
!unzip -o DelayedFlights.zip -d data

import pandas as pd

# Load Dataset
df_raw = pd.read_csv('data/airlinedelaycauses_DelayedFlights.csv')

# Copy Dataset for Preprocessing
df = df_raw.copy()

# Preview Dataset
df_raw.head()

## Dataset Overview

### DOT’S Air Travel Consumer Report:
The U.S. Department of Transportation’s (DOT) Bureau of Transportation Statistics (BTS) tracks the **on-time performance** of domestic flights operated by large air carriers.
- DOT provides **monthly summary information** on the number of on-time, delayed, cancelled and diverted flights.
- BTS collects details on the **causes of flight delays** and releases summary statistics and raw data.


## Summary Statistics

In [ ]:
# Understand the dataset
df_raw.info()
df_raw.shape

In [ ]:
# Basic Summary Statistics
df_raw.describe().T

### Understanding the Dataset
The initial output upon loading the *full raw dataset* includes:
- **1,936,758 rows (flights) x 30 columns (features)**
- The columns consist of both numeric (`int64`, `float64`) and categorical (`object`) features, including:
  - 14 float variables (e.g., `DepTime`, `ArrTime`, `DepDelay`, `ArrDelay`)
  - 11 integer variables (e.g., `Year`, `Month`, `DayofWeek`, `FlightNum`)
  - 5 object variables (e.g., `UniqueCarrier`, `Origin`, `Dest`)  

According to PA4, the dataset *should* include:
- Flight delay metrics for **1,247,486** different flights.
- **30 different features**, both numerical and categorical.

### Understanding the Record Discrepancy
The difference between the ~1.94 million and ~1.25 million flight records is explained by *dataset scope*.
- The full raw dataset (**1,936,758 rows**) includes **all scheduled flights** in 2008 - whether they were on time, delayed, cancelled, or diverted.
- Only a subset of the flight records (**1,247,488 rows**) contain complete **delay-related data** (`CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAircraftDelay`).
  - These represent flights that actually experienced a *delay event*, which is the primary focus of this data analysis.

As a result, all preprocessing and subsequent visualizations are performed on this ~1.25M delayed-flight subset to ensure meaningful data analysis.

## Preprocessing

In [ ]:
# PREPROCESSING

# Drop redundant/irrelevant columns
df = df.drop(columns=['Unnamed: 0', 'Year', 'FlightNum', 'TailNum', 'CancellationCode'], errors='ignore')

# Remove cancelled or diverted flights
df = df[(df['Cancelled'] == 0) & (df['Diverted'] == 0)]

# Identify records with complete delay-cause data
DelayCause = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
df = df.dropna(subset=DelayCause)

# Drop rows missing essential delay information
df = df.dropna(subset=['ArrDelay'])

# Fill missing delay-cause values (NaN) with 0
df[DelayCause] = df[DelayCause].fillna(0)

# Clip negative delay values (representing early arrivals) to 0
df['ArrDelay'] = df['ArrDelay'].clip(lower=0)

# Derive new time-based features for later visualization
df['DepHour'] = (df['DepTime'] // 100).astype(int)
df['ArrHour'] = (df['ArrTime'] // 100).astype(int)

# Convert arrival/departure delays from minutes to hours
df['ArrDelayHours'] = (df['ArrDelay'] / 60).astype('float64')
df['DepDelayHours'] = (df['DepDelay'] / 60).astype('float64')

# Create distance categories (in miles)
df['DistanceGroup'] = pd.cut(
    df['Distance'],
    bins=[0, 500, 1000, 2000, 3000, 5000],
    labels=['<500', '500–1000', '1000–2000', '2000–3000', '3000–5000']
)

# Reset DataFrame index to ensure clean row alignment
df = df.reset_index(drop=True)

# Verify structure after preprocessing
df.info()
df.shape

### Data Preprocessing
## 🧹 Data Preprocessing

*Before visualizing flight delay trends, the dataset required preprocessing to ensure data analysis focuses only on valid, delayed flight records.*

### 1. Removed redundant and irrelevant columns
- The first column, `Unnamed: 0`, is an index column automatically generated during export and does not represent a meaningful feature.
- Dropped `Year` since all records were from 2008 (constant value), offering no variance for analysis.
- `FlightNum` and `TailNum` provide no meaningful information for data analysis.

### 2. Excluded cancelled or diverted flights
- Removed flights where `Cancelled = 1` or `Diverted = 1`.  
- These records do not have valid arrival/departure data, which is essential for delay analysis.

### 3. Retained only records with complete delay-cause data
- Filtered to include only rows where all five delay cause fields were present:  
  `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAircraftDelay`.  
- This isolates the ~1.25 million delayed-flight subset that contains full delay-cause information — the focus of this data analysis.

### 4. Dropped missing arrival delay values
- `ArrDelay` had 1,928,371 non-null records out of 1,936,758.  
- Rows missing `ArrDelay` were removed to ensure valid arrival delay metrics for analysis.  
- `DepDelay` was already complete, so no removal was necessary.

### 5. Filled missing delay-cause values with zero
- Any remaining `NaN` values in delay-cause columns were replaced with `0`
- NaN indicated *no delay* from that cause (e.g., `WeatherDelay`).

### 6. Clipped negative arrival delay values
- Negative values in `ArrDelay` represent early arrivals (e.g., `-109` = 109 minutes early).  
- To focus purely on delays, these were clipped to `0`, indicating no delay.  
- `DepDelay` contained no negative values, so no adjustment was required.

### 7. Derived new time-based features
- Created `DepHour` and `ArrHour` by converting scheduled departure/arrival times (e.g., `1530`) to hour bins (e.g., `15`).  
- This facilitates later visualization of delay patterns by time of day.

### 8. Reset DataFrame index
- Reset the index after all filtering steps to maintain continuous row alignment.

### Summary
After preprocessing:
- Rows reduced from **1,936,758** to **1,247,488**.
- Columns remained **30**, with two new derived features (`DepHour`, `ArrHour`).  
- Resulting dataset represents all recorded data for **delayed flights in 2008** that experienced measurable delay causes.


In [ ]:
# Basic summary statistics after preprocessing
df.describe().T

In [ ]:
print(f"Raw dataset shape: {df_raw.shape}")
print(f"Cleaned dataset shape: {df.shape}")
print(f"Rows removed during preprocessing: {df_raw.shape[0] - df.shape[0]:,}")

In [ ]:
# Delays greater than 24 hours
(df['ArrDelay'] > 1440).sum(), (df['DepDelay'] > 1440).sum(), (df['CarrierDelay'] > 1440).sum()

## Analyze Dataset

In [ ]:
# Experiment with basic charts/visualizations

In [ ]:
df.groupby('DepHour')['ArrDelay'].mean().plot(kind='bar', figsize=(8,4))
plt.title("Average Arrival Delay by Departure Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Average Arrival Delay (minutes)")
plt.show()

In [ ]:
df['DepHour'].value_counts().sort_index().plot(kind='bar', figsize=(8,4))
plt.title("Number of Delayed Flights by Departure Hour")
plt.xlabel("Hour of Day (Departure)")
plt.ylabel("Number of Flights")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
df.groupby('DepHour')[['ArrDelayHours', 'DepDelayHours']].mean().plot(kind='bar')
plt.title('Average Arrival and Departure Delays by Hour')
plt.xlabel('Hour of Day (Departure)')
plt.ylabel('Average Delay (Hours)')
plt.legend(['Arrival Delay', 'Departure Delay'])
plt.show()

In [ ]:
# Create pivot table for average arrival delay by weekday and hour
pivot = df.pivot_table(
    values='ArrDelayHours',
    index='DepHour',       # rows = hour of day
    columns='DayOfWeek',   # columns = day of week
    aggfunc='mean'
)

# Rename day columns for clarity
pivot.columns = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Plot heatmap
plt.figure(figsize=(8,6))
sns.heatmap(pivot, cmap='coolwarm', annot=False, cbar_kws={'label': 'Average Arrival Delay (hours)'})
plt.title('Average Arrival Delay by Hour and Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')
plt.show()

In [ ]:
# Pivot table: average arrival delay grouped by month and weekday
pivot = df.pivot_table(
    values='ArrDelayHours',
    index='Month',        # months as rows
    columns='DayOfWeek',  # weekdays as columns
    aggfunc='mean'
)

# Replace numeric weekdays (1–7) with labels
pivot.columns = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Replace numeric months (1–12) with short names
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
pivot.index = month_labels

# Plot heatmap
plt.figure(figsize=(9,6))
sns.heatmap(pivot, cmap='YlOrRd', annot=False, cbar_kws={'label': 'Avg Arrival Delay (hours)'})
plt.title('Average Arrival Delay by Month and Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Month')
plt.show()

In [ ]:
# Count total number of delayed flights per day of week
delays_by_day = df['DayOfWeek'].value_counts().sort_index()

# Replace numeric day codes (1–7) with weekday names
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
delays_by_day.index = days

# Plot
plt.figure(figsize=(8,4))
delays_by_day.plot(kind='bar', color='steelblue')

plt.title("Total Number of Delayed Flights by Day of Week")
plt.xlabel("Day of Week")
plt.ylabel("Number of Delayed Flights")
plt.show()

In [ ]:
# Sum delay causes by month (for all flights)
monthly_cause = df_raw.groupby('Month')[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'LateAircraftDelay']].sum()

# Calculate percentage of total delay minutes
monthly_percent = monthly_cause.div(monthly_cause.sum().sum(), axis=0) * 100

# Convert month numbers to names
monthly_percent = monthly_percent.reset_index()
monthly_percent['Month'] = monthly_percent['Month'].replace({
    1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
    7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'
})

# Plot
fig = px.area(
    monthly_percent,
    x='Month',
    y=['CarrierDelay', 'WeatherDelay', 'NASDelay', 'LateAircraftDelay'],
    title='Delay Causes by Month',
    labels={'value':'% of Total Delay Minutes', 'variable':'Delay Cause'}
)

fig.show()

In [ ]:
(df[['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']] > 0).sum()

In [ ]:
avg_delays = df[delayed].mean().sort_values(ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(x=avg_delays.index, y=avg_delays.values)
plt.title("Average Delay per Flight by Cause")
plt.xlabel("Cause of Delay")
plt.ylabel("Average Delay (minutes)")
plt.show()

In [ ]:
# Calculate average delay minutes per cause for each month
monthly_avg = (
    df.groupby('Month')[['CarrierDelay','WeatherDelay','NASDelay','LateAircraftDelay']]
    .mean()
    .reset_index()
)

# Replace month numbers with names for readability
monthly_avg['Month'] = monthly_avg['Month'].replace({
    1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
    7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'
})

# Plot all 5 causes in one interactive line chart
fig = px.line(
    monthly_avg,
    x='Month',
    y=['CarrierDelay','WeatherDelay','NASDelay','LateAircraftDelay'],
    markers=True,
    title='Average Delay per Flight by Month and Cause',
    labels={'value':'Average Delay (minutes)', 'variable':'Delay Cause'},
)

fig.show()

In [ ]:
avg_delay_carrier = df.groupby('UniqueCarrier')[['ArrDelay','DepDelay']].mean().sort_values('ArrDelay', ascending=False)
avg_delay_carrier.plot(kind='bar', figsize=(12,6))
plt.title("Average Arrival & Departure Delays by Airline")
plt.ylabel("Arrival/Departure Delay (minutes)")
plt.show()

In [ ]:
# Count delayed flights per day of week
delays_per_day = df_raw['DayOfWeek'].value_counts().sort_index()

# Convert to percentage
delay_percent = (delays_per_day / delays_per_day.sum()) * 100

# Create DataFrame for plotting
delay_data = delay_percent.reset_index()
delay_data.columns = ['DayOfWeek', 'PercentDelayed']

# Replace numeric day codes (1–7) with labels
delay_data['DayOfWeek'] = delay_data['DayOfWeek'].replace({
    1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat', 7:'Sun'
})

# Plot
fig = px.bar(
    delay_data,
    x='DayOfWeek',
    y='PercentDelayed',
    title='Percentage of Total Delayed Flights by Day of Week',
    labels={'PercentDelayed':'% of Delayed Flights', 'DayOfWeek':'Day of Week'},
    color='PercentDelayed',
    color_continuous_scale='Blues'
)

fig.update_layout(template='simple_white', title_x=0.5)
fig.show()

In [ ]:
top_airports = df.groupby('Origin')['ArrDelayHours'].mean().sort_values(ascending=False).head(10)
top_airports.plot(kind='bar')
plt.title("Top 10 Airports by Average Arrival Delay")
plt.ylabel("Average Delay (hours)")
plt.show()

In [ ]:
# Keep only realistic delays (under 24 hours)
df_filtered = df[df['ArrDelayHours'] <= 24]

# Scatter: distance vs arrival delay hours
fig = px.scatter(
    df_filtered,
    x='Distance',
    y='ArrDelayHours',
    opacity=0.3,
    trendline='ols',
    title='Relationship Between Flight Distance and Arrival Delay (≤ 24 Hours)',
    labels={'Distance':'Flight Distance (miles)', 'ArrDelayHours':'Arrival Delay (hours)'}
)
fig.show()

In [ ]:
# Count number of flights in each range
distance_counts = df['DistanceGroup'].value_counts().sort_index().reset_index()
distance_counts.columns = ['Distance Range', 'Flight Count']

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=distance_counts, x='Distance Range', y='Flight Count')
plt.title('Number of Flights by Distance Range')
plt.xlabel('Flight Distance (miles)')
plt.ylabel('Number of Flights')
plt.show()

# **INTERACTIVE VISUALIZATION DASHBOARD**

## **Visualization 1**

In [ ]:
# 1

### Conclusion

## **Visualization 2**

In [ ]:
# 2

### Conclusion

## **Visualization 3**

In [ ]:
# 3

### Conclusion

## **Visualization 4**

In [ ]:
# 4

### Conclusion



## **Visualization 5**

In [ ]:
# 5

### Conclusion